In [9]:
import numpy as np
from scipy.fftpack import fft2, ifft2
from scipy.integrate import solve_ivp
from scipy.linalg import kron

# Define global parameters
D1 = 0.1
D2 = 0.1
beta = 1
tspan = np.arange(0, 4.5, 0.5)  # Time span
Lx, Ly = 20, 20
nx, ny = 64, 64
N = nx * ny
T = 4

# Define spatial grid and initial conditions for periodic boundaries
x2 = np.linspace(-Lx / 2, Lx / 2, nx + 1)
x = x2[:nx]
y2 = np.linspace(-Ly / 2, Ly / 2, ny + 1)
y = y2[:ny]
X, Y = np.meshgrid(x, y)

# Define spectral k values
kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, nx / 2), np.arange(-nx / 2, 0)))
kx[0] = 1e-6
ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, ny / 2), np.arange(-ny / 2, 0)))
ky[0] = 1e-6
KX, KY = np.meshgrid(kx, ky)
K = KX**2 + KY**2

# Spiral initial conditions
m = 1
u = np.tanh(np.sqrt(X**2 + Y**2)) * np.cos(m * np.angle(X + 1j * Y) - np.sqrt(X**2 + Y**2))
v = np.tanh(np.sqrt(X**2 + Y**2)) * np.sin(m * np.angle(X + 1j * Y) - np.sqrt(X**2 + Y**2))
ut = fft2(u)
vt = fft2(v)
uvt0 = np.hstack([(ut.reshape(N)), (vt.reshape(N))])

# Define the RHS for periodic boundaries
def spc_rhs(t, uvt, nx, ny, N, D1, D2, beta, K):
    utc = uvt[:N].reshape((nx, ny))
    vtc = uvt[N:].reshape((nx, ny))
    u = np.real(ifft2(utc))
    v = np.real(ifft2(vtc))
    A = u**2 + v**2
    lam = 1 - A
    ome = -beta * A

    rhs_u = (-D1 * K * utc + fft2(lam * u - ome * v)).reshape(N)
    rhs_v = (-D2 * K * vtc + fft2(ome * u + lam * v)).reshape(N)
    return np.hstack((rhs_u, rhs_v))

# Solve using FFT (Periodic)
uvtsol = solve_ivp(spc_rhs, [0, T], uvt0, t_eval=tspan, args=(nx, ny, N, D1, D2, beta, K), method='RK45')
A1 = uvtsol.y


# Chebyshev differentiation matrix
def chebb(N):
    if N == 0:
        D = 0
        x = 1
    else:
        n = np.arange(0, N + 1)
        x = np.cos(np.pi * n / N).reshape(N + 1, 1)
        c = (np.hstack(([2], np.ones(N - 1), [2])) * (-1)**n).reshape(N + 1, 1)
        X = np.tile(x, (1, N + 1))
        dX = X - X.T
        D = np.dot(c, 1. / c.T) / (dX + np.eye(N + 1))
        D -= np.diag(np.sum(D.T, axis=0))
    return D, x.flatten()

# Define spatial grid and initial conditions for no-flux boundaries
N = 30
D, x = chebb(N)
D[N, :] = 0
D[0, :] = 0
Dxx = np.dot(D, D) / (10**2)
y = x
N2 = (N + 1)**2
I = np.eye(len(Dxx))
L = kron(I, Dxx) + kron(Dxx, I)

X, Y = np.meshgrid(x, y)
X, Y = X * 10, Y * 10
u = np.tanh(np.sqrt(X**2 + Y**2)) * np.cos(m * np.angle(X + 1j * Y) - np.sqrt(X**2 + Y**2))
v = np.tanh(np.sqrt(X**2 + Y**2)) * np.sin(m * np.angle(X + 1j * Y) - np.sqrt(X**2 + Y**2))
uv0 = np.hstack([(u.reshape(N2)), (v.reshape(N2))])

# Define the RHS for no-flux boundaries
def RD(t, uv, L, N2, D1, D2, beta):
    u = uv[:N2]
    v = uv[N2:]
    A2 = u**2 + v**2
    lam = 1 - A2
    ome = -beta * A2

    rhs_u = D1 * np.dot(L, u) + lam * u - ome * v
    rhs_v = D2 * np.dot(L, v) + ome * u + lam * v
    return np.hstack((rhs_u, rhs_v))

# Solve using Chebyshev (No-Flux)
uvsol = solve_ivp(RD, [0, T], uv0, t_eval=tspan, args=(L, N2, D1, D2, beta), method='RK45')
A2 = uvsol.y
print("A2:", A2)
print("A1", A1)

A2: [[ 0.70358468  0.27678435 -0.21775865 ... -0.79689015 -0.40972859
   0.07776933]
 [ 0.73241275  0.47188952  0.07344742 ... -0.96577657 -0.78500366
  -0.4261521 ]
 [ 0.81058026  0.37605887 -0.11123233 ... -0.84008598 -0.49565779
  -0.03085913]
 ...
 [ 0.58562756  0.91352592  0.97914313 ... -0.50294695 -0.84298442
  -0.97634716]
 [ 0.6808609   0.87018536  0.97997159 ... -0.16453512 -0.5878894
  -0.88455009]
 [ 0.71061143  0.96093661  0.97601586 ... -0.60413504 -0.91222169
  -0.99697897]]
A1 [[ 24.94003847  +0.j          12.73268299  +0.j
   -1.38095598  +0.j         ... -64.02389647  +0.j
  -67.76356741  +0.j         -61.18058974  +0.j        ]
 [-18.55666362 -58.16631091j -42.51586944 -46.91292244j
  -60.80795253 -25.74803902j ... -26.39439597+113.08288984j
    6.86544434+123.00045628j  41.4436393 +110.05531209j]
 [-16.04755868 +32.82798293j -22.03971648 -45.79777396j
  -23.23089505-104.14171583j ... -25.03391682 -92.65273136j
  -29.2936105  -40.95948731j -31.3712619  +15.69868908j]